<a href="https://colab.research.google.com/github/cocoisland/DS-Unit-2-Sprint-4-Model-Validation/blob/master/pump.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install kaggle

In [3]:
from google.colab import drive
drive.mount('/content/drive')
%env KAGGLE_CONFIG_DIR=/content/drive/My Drive/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
env: KAGGLE_CONFIG_DIR=/content/drive/My Drive/


In [4]:
!kaggle  competitions download -c ds1-predictive-modeling-challenge

sample_submission.csv: Skipping, found more recently modified local copy (use --force to force download)
test_features.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
train_labels.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
train_features.csv.zip: Skipping, found more recently modified local copy (use --force to force download)


In [5]:
!unzip train_features.csv.zip
!unzip train_labels.csv.zip
!unzip test_features.csv.zip



Archive:  train_features.csv.zip
replace train_features.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: N
Archive:  train_labels.csv.zip
replace train_labels.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: train_labels.csv        
Archive:  test_features.csv.zip
replace test_features.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: test_features.csv       


In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import sklearn.feature_selection as fe
from sklearn.feature_selection import SelectPercentile, SelectKBest, f_classif
from sklearn.feature_selection import RFECV

from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler

from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_validate

from mlxtend.plotting import plot_decision_regions
#import category_encoders as ce


In [7]:
# DO NOT USE
# date as index
df_train = pd.read_csv('train_features.csv',index_col='date_recorded', 
              parse_dates=True, infer_datetime_format=True)
df_test = pd.read_csv('test_features.csv', index_col='date_recorded',
                     parse_dates=True, infer_datetime_format=True)
df_label = pd.read_csv('train_labels.csv')

df_train.shape, df_test.shape, df_label.shape


((59400, 39), (14358, 39), (59400, 2))

In [31]:
# id as index
df_train = pd.read_csv('train_features.csv',index_col='id')
df_test = pd.read_csv('test_features.csv', index_col='id')
df_label = pd.read_csv('train_labels.csv')
df_sample = pd.read_csv('sample_submission.csv')

df_train.shape, df_test.shape, df_label.shape, df_sample.shape


((59400, 39), (14358, 39), (59400, 2), (14358, 2))

In [32]:
X_train_val, X_test, y_train_val, y_test = train_test_split(
        df_train, df_label, test_size=0.25, random_state=42, shuffle=True)

X_train_val.shape, X_test.shape, y_train_val.shape, y_test.shape

((44550, 39), (14850, 39), (44550, 2), (14850, 2))

In [0]:
# majority baseline
# balanced data - logistic class_weight = 
#            (None, accuracy = 0.55),
#            (balanced,accuracy=0.39-0.4)


df_label['status_group'].value_counts(normalize=True)

In [47]:
pd.set_option('display.max_columns', None)

#df_str=X_train_val.select_dtypes(exclude=['int','float']).copy()
df_str=X_train_val.select_dtypes(object).copy()
df_str.head(3)


,date_recorded,funder,installer,wpt_name,basin,subvillage,region,lga,ward,public_meeting,recorded_by,scheme_management,scheme_name,permit,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
33935,2011-03-08,Government Of Tanzania,Ce,Ccm,Wami / Ruvu,Kilangawageni,Pwani,Bagamoyo,Mbwewe,True,GeoData Consultants Ltd,VWC,Kwaruhombo wa,True,ksb,submersible,submersible,vwc,user-group,pay per bucket,per bucket,salty,salty,insufficient,insufficient,machine dbh,borehole,groundwater,communal standpipe,communal standpipe
49654,2013-02-16,Dwsp,DWE,Mwamahonza,Lake Victoria,Mwamahala,Mwanza,Magu,Nkungulu,NaN,GeoData Consultants Ltd,WUG,NaN,False,nira/tanira,nira/tanira,handpump,wug,user-group,unknown,unknown,soft,good,enough,enough,shallow well,shallow well,groundwater,hand pump,hand pump
39287,2011-08-13,Dasip,DASIP,Chanongu,Lake Victoria,Chanongu,Mwanza,Kwimba,Wala,True,GeoData Consultants Ltd,VWC,NaN,True,india mark ii,india mark ii,handpump,vwc,user-group,never pay,never pay,soft,good,enough,enough,machine dbh,borehole,groundwater,hand pump,hand pump


In [157]:
X_train_val.water_quality.value_counts()

soft                  38064
salty                  3674
unknown                1398
milky                   620
coloured                373
salty abandoned         249
fluoride                159
fluoride abandoned       13
Name: water_quality, dtype: int64

In [158]:
X_train_val.quality_group.value_counts()

good        38064
salty        3923
unknown      1398
milky         620
colored       373
fluoride      172
Name: quality_group, dtype: int64

In [0]:
# As observations increase, more features will fail to pass the threshold
# At 50,000+ observation, passing threshold drops to 0.1

#remove all features that are either one or zero (on or off) in more than 80% of the samples.
#threshold = 0.8 * (1- .8)

import sklearn.feature_selection as fe

def variance_threshold_selector(data, threshold=0.01):
    selector = fe.VarianceThreshold(threshold)
    selector.fit(data)
    return data[data.columns[selector.get_support(indices=True)]]

In [58]:
df_test.isnull().sum()

amount_tsh                  0
date_recorded               0
funder                    783
gps_height                  0
installer                 788
longitude                   0
latitude                    0
wpt_name                    0
num_private                 0
basin                       0
subvillage                 94
region                      0
region_code                 0
district_code               0
lga                         0
ward                        0
population                  0
public_meeting            785
recorded_by                 0
scheme_management         939
scheme_name              6839
permit                    663
construction_year           0
extraction_type             0
extraction_type_group       0
extraction_type_class       0
management                  0
management_group            0
payment                     0
payment_type                0
water_quality               0
quality_group               0
quantity                    0
quantity_g

In [0]:
X_.quality_group.replace('unknown','unkwn_qual_grp')

In [0]:
def wrangle(X):
  
  # get all continuous value column
  X_num = X._get_numeric_data()
  
  
  # funder
  X.funder = X.funder.replace(np.nan, 'missing')
  X.funder = X.funder.replace('0', 'missing')
  X.funder = X.funder.replace('other','other_funder')
  X_dummies = pd.get_dummies(X.funder)
  dummy_filter=variance_threshold_selector(X_dummies)
  X_num = X_num.join(dummy_filter, how='inner')
  
  # quality_group
  X.quality_group=X.quality_group.replace('unknown','unkwn_qual_grp')
  X_dummies = pd.get_dummies(X.quality_group)
  dummy_filter=variance_threshold_selector(X_dummies)
  X_num = X_num.join(dummy_filter, how='inner')
  
  # quantity
  X.quantity=X.quantity.replace('unknown','unknown_quantity')
  X_dummies = pd.get_dummies(X.quantity)
  dummy_filter=variance_threshold_selector(X_dummies)
  X_num = X_num.join(dummy_filter, how='inner')
  
  # source
  X.source_type=X.source_type.replace('other','other_source')
  X_dummies = pd.get_dummies(X.source_type)
  dummy_filter=variance_threshold_selector(X_dummies)
  X_num = X_num.join(dummy_filter, how='inner')
  
  '''# source_class
  X.source_class.replace('unknown','unknown_source_class')
  X_dummies = pd.get_dummies(X.source_class)
  dummy_filter=variance_threshold_selector(X_dummies)
  X_num = X_num.join(dummy_filter, how='inner')'''
  
  # waterpoint_type
  X.waterpoint_type=X.waterpoint_type.replace('other','other_waterpoint')
  X_dummies = pd.get_dummies(X.waterpoint_type)
  dummy_filter=variance_threshold_selector(X_dummies)
  X_num = X_num.join(dummy_filter, how='inner')
  
  
  #y = y.replace(['non functional','functional','functional needs repair'],[1,2,3])
  #y = y.values
  
  
  return X_num
  

In [0]:
# NOT USE - for understanding variance_threshold only
# same as variance threshold function
def noise_filter(df, col, threshold=0.01):
  X_dummies=pd.DataFrame()
  X_dummies[col]=df[col].value_counts(normalize=True)

  for noise_cat in X_dummies.loc[(X_dummies[col] < threshold)].index.values:
    df[col].replace(noise_cat, 'noise', inplace=True)

  #df_str['funder'].replace('0', 'noise', inplace=True)
  #df_str['funder'].replace(np.nan,'noise' ,inplace=True)

  return pd.get_dummies(df[col])

dummy_filter = noise_filter(df_str, 'funder' ,0.01)
dummy_filter=dummy_filter.drop(['0','noise'], axis=1)
dummy_filter.shape

In [170]:
# Baseline metric - numeric only - test_score=0.55
X = wrangle(X_train_val)
y = y_train_val.status_group.values


scores = cross_validate(LogisticRegression(solver='lbfgs',class_weight=None), X, y, 
                        scoring='accuracy', cv=3,
                        return_train_score=True, return_estimator=True)

pd.DataFrame(scores)

,estimator,fit_time,score_time,test_score,train_score
0,"LogisticRegression(C=1.0, class_weight=None, d...",2.138110,0.033144,0.676318,0.671740
1,"LogisticRegression(C=1.0, class_weight=None, d...",2.509811,0.033069,0.671111,0.673300
2,"LogisticRegression(C=1.0, class_weight=None, d...",2.457643,0.032952,0.681123,0.682704


In [0]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_val, y_pred)

In [19]:
X_test.shape, df_test.shape

((14850, 39), (14358, 39))

In [0]:
y_test.shape, df

In [0]:
# Define an estimator and param_grid
from sklearn.feature_selection import SelectPercentile, SelectKBest, f_classif,mutual_info_classif

#X = wrangle(df_test)

X = wrangle(X_train_val)
#y = y_test.status_group.values


pipe = make_pipeline(
    RobustScaler(), 
    #fe.SelectKBest(mutual_info_classif), #slow 0.58
    fe.SelectKBest(f_classif),
    LogisticRegression(solver='lbfgs'))

param_grid = {
    'selectkbest__k': ['all'],
    'logisticregression__class_weight': [None, 'balanced'],
    'logisticregression__C': [.01, .1, 1.0, 10.0]
}

# Fit on the train set, with grid search cross-validation
gs = GridSearchCV(pipe, param_grid=param_grid, cv=3, 
                  scoring='accuracy', return_train_score=True,
                  verbose=1)



# Need help here. gs.predict(X) can't predict without y target.

In [76]:
#X = wrangle(X_test)
X=wrangle(df_test)
#gs.best_estimator_.predict(X)
gs.predict(X)

NotFittedError: ignored

In [136]:
X = wrangle(X_train_val)
y = y_train_val.status_group

gs.fit(X, y)
validation_score = gs.best_score_
print()
print('Cross-Validation Score:', validation_score)
print()
print('Best estimator:', gs.best_estimator_)
print()

results = pd.DataFrame(gs.cv_results_)
print('Best result from grid search of {} parameter combinations'.format(len(results)))
results.sort_values(by='rank_test_score').head(3)

Fitting 3 folds for each of 8 candidates, totalling 24 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  24 out of  24 | elapsed:  1.0min finished


()
('Cross-Validation Score:', 0.6450280583613917)
()
('Best estimator:', Pipeline(memory=None,
     steps=[('robustscaler', RobustScaler(copy=True, quantile_range=(25.0, 75.0), with_centering=True,
       with_scaling=True)), ('selectkbest', SelectKBest(k='all', score_func=<function f_classif at 0x7f7bcd442d70>)), ('logisticregression', LogisticRegression(C=1.0, class_weight=None, dual=False, fit_...enalty='l2', random_state=None, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False))]))
()
Best result from grid search of 8 parameter combinations


,mean_fit_time,mean_score_time,mean_test_score,mean_train_score,param_logisticregression__C,param_logisticregression__class_weight,param_selectkbest__k,params,rank_test_score,split0_test_score,split0_train_score,split1_test_score,split1_train_score,split2_test_score,split2_train_score,std_fit_time,std_score_time,std_test_score,std_train_score
4,2.507492,0.046168,0.645028,0.645399,1,None,all,"{u'selectkbest__k': u'all', u'logisticregressi...",1,0.652077,0.649517,0.640471,0.645118,0.642535,0.641561,0.030354,0.000180,0.005055,0.003254
6,2.480535,0.046090,0.642694,0.642851,10,None,all,"{u'selectkbest__k': u'all', u'logisticregressi...",2,0.646286,0.643254,0.637912,0.643098,0.643882,0.642201,0.023110,0.000117,0.003520,0.000464
0,2.349126,0.033820,0.641841,0.642166,0.01,None,all,"{u'selectkbest__k': u'all', u'logisticregressi...",3,0.645142,0.642412,0.638249,0.643771,0.642131,0.640315,0.183935,0.000130,0.002821,0.001422


In [88]:
#Which features were selected?


selector = gs.best_estimator_.named_steps['selectkbest']
all_names = X.columns.values
selected_mask = selector.get_support()
selected_names = all_names[selected_mask]
unselected_names = all_names[~selected_mask]

print('Features selected:')
for name in selected_names:
    print(name)

print('\n')
print('Features not selected:')
for name in unselected_names:
    print(name)


Features selected:
amount_tsh
gps_height
longitude
latitude
num_private
region_code
district_code
population
construction_year


Features not selected:


In [92]:
from sklearn.ensemble import RandomForestRegressor as RFR

X=wrangle(X_train_val)
names = X.columns

y = y_train_val.status_group.values

y = pd.DataFrame(y).replace(['non functional','functional','functional needs repair'],[1,2,3])
y = y.values

# Fitting a random forest regression
rfr = RFR().fit(X, y)

# Creating scores from feature_importances_ ranking (some randomness here)
rfr_scores = pd.Series(data=rfr.feature_importances_, name='RFR', index=names)

rfr_scores

/usr/local/lib/python2.7/dist-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:12: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  if sys.path[0] == '':


amount_tsh           0.046799
gps_height           0.112360
longitude            0.321401
latitude             0.314932
num_private          0.002886
region_code          0.020020
district_code        0.024160
population           0.086345
construction_year    0.071098
Name: RFR, dtype: float64

In [94]:
X.head(3)

,amount_tsh,gps_height,longitude,latitude,num_private,region_code,district_code,population,construction_year
id,,,,,,,,,
33935,20.0,330,38.123839,-6.087137e+00,0,6,1,80,1972
49654,0.0,0,0.000000,-2.000000e-08,0,17,1,0,0
39287,0.0,0,33.312321,-2.814100e+00,0,19,4,0,0


In [93]:
from sklearn.feature_selection import RFECV
from sklearn.linear_model import Lasso

X= wrangle(X_train_val)

y= y_train_val.status_group

#y = y.replace(['non functional','functional','functional needs repair'],[1,2,3])
#y = y.values

X_scaled = RobustScaler().fit_transform(X)

# Lasso can't handle multi-class label.
#rfe = RFECV(Lasso(), scoring='accuracy', cv=3)
rfe = RFECV(LogisticRegression(), scoring='accuracy', cv=3)
X_subset = rfe.fit_transform(X_scaled, y)

all_names = X.columns
selected_mask = rfe.support_
selected_names = all_names[selected_mask]
unselected_names = all_names[~selected_mask]

print('Features selected:')
for name in selected_names:
    print(name)

print()
print('Features not selected:')
for name in unselected_names:
    print(name)


/usr/local/lib/python2.7/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Features selected:
amount_tsh
gps_height
longitude
latitude
num_private
region_code
district_code
population
construction_year
()
Features not selected:


In [29]:
X,y=wrangle(df_test, y_test)
X.shape, y.shape

((14358, 26), (14850,))

In [0]:
# Predict with X_test features
X=wrangle(df_test)
y_pred = gs.predict(X)

# Compare predictions to y_test labels
#test_score = accuracy_score(y_test.status_group, y_pred)
#print('Test Score:', test_score)

In [15]:
#submission as format as df_sample
y_submission = y_test.copy()
y_submission['status_group']=y_pred
y_submission.shape

(14850, 2)

In [0]:
from google.colab import files

y_submission.to_csv('data.csv', index=False)
files.download("data.csv")